# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 20 2022 11:35AM,249073,16,SHL-8671207,"SHL Pharma, LLC",Released
1,Oct 20 2022 11:09AM,249072,10,8671750,"Citieffe, Inc.",Released
2,Oct 20 2022 11:02AM,249070,15,30011555,"Alliance Pharma, Inc.",Executing
3,Oct 20 2022 11:02AM,249070,15,30011552,"Alliance Pharma, Inc.",Released
4,Oct 20 2022 11:02AM,249070,15,30011553,"Alliance Pharma, Inc.",Executing
5,Oct 20 2022 11:02AM,249070,15,30011554,"Alliance Pharma, Inc.",Released
6,Oct 20 2022 11:02AM,249070,15,30011558,"Alliance Pharma, Inc.",Released
7,Oct 20 2022 11:02AM,249070,15,30011556,"Alliance Pharma, Inc.",Released
8,Oct 20 2022 11:02AM,249070,15,30011559,"Alliance Pharma, Inc.",Released
9,Oct 20 2022 11:02AM,249070,15,30011557,"Alliance Pharma, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,249068,Released,10
26,249070,Executing,14
27,249070,Released,28
28,249072,Released,1
29,249073,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249066,NaN,NaN,1.0
249068,NaN,NaN,10.0
249070,NaN,14.0,28.0
249072,NaN,NaN,1.0
249073,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0.0,0.0,1.0
248979,0.0,0.0,2.0
249001,0.0,4.0,4.0
249002,0.0,1.0,1.0
249006,4.0,11.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248896,0,0,1
248979,0,0,2
249001,0,4,4
249002,0,1,1
249006,4,11,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,0,0,1
1,248979,0,0,2
2,249001,0,4,4
3,249002,0,1,1
4,249006,4,11,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248896,,,1
1,248979,,,2
2,249001,,4,4
3,249002,,1,1
4,249006,4,11,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC"
1,Oct 20 2022 11:09AM,249072,10,"Citieffe, Inc."
2,Oct 20 2022 11:02AM,249070,15,"Alliance Pharma, Inc."
44,Oct 20 2022 10:56AM,249068,10,Bio-PRF
54,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC"
55,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.
69,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.
70,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc."
71,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation
91,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",,,1
1,Oct 20 2022 11:09AM,249072,10,"Citieffe, Inc.",,,1
2,Oct 20 2022 11:02AM,249070,15,"Alliance Pharma, Inc.",,14,28
3,Oct 20 2022 10:56AM,249068,10,Bio-PRF,,,10
4,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",,,1
5,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,,9,5
6,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,,,1
7,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",,1,
8,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation,1,17,2
9,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd",3,38,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",1,,
1,Oct 20 2022 11:09AM,249072,10,"Citieffe, Inc.",1,,
2,Oct 20 2022 11:02AM,249070,15,"Alliance Pharma, Inc.",28,14,
3,Oct 20 2022 10:56AM,249068,10,Bio-PRF,10,,
4,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1,,
5,Oct 20 2022 10:17AM,249064,10,Eywa Pharma Inc.,5,9,
6,Oct 20 2022 10:13AM,249065,10,Eywa Pharma Inc.,1,,
7,Oct 20 2022 9:36AM,249063,21,"NBTY Global, Inc.",,1,
8,Oct 20 2022 8:55AM,249054,10,ISDIN Corporation,2,17,1
9,Oct 20 2022 8:46AM,249053,19,"AdvaGen Pharma, Ltd",10,38,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",1,,
1,Oct 20 2022 11:09AM,249072,10,"Citieffe, Inc.",1,,
2,Oct 20 2022 11:02AM,249070,15,"Alliance Pharma, Inc.",28,14,
3,Oct 20 2022 10:56AM,249068,10,Bio-PRF,10,,
4,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Oct 20 2022 11:09AM,249072,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Oct 20 2022 11:02AM,249070,15,"Alliance Pharma, Inc.",28.0,14.0,NaN
3,Oct 20 2022 10:56AM,249068,10,Bio-PRF,10.0,NaN,NaN
4,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 20 2022 11:35AM,249073,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Oct 20 2022 11:09AM,249072,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Oct 20 2022 11:02AM,249070,15,"Alliance Pharma, Inc.",28.0,14.0,0.0
3,Oct 20 2022 10:56AM,249068,10,Bio-PRF,10.0,0.0,0.0
4,Oct 20 2022 10:34AM,249066,10,"Coronado Aesthetics, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2490388,25.0,53.0,5.0
15,249070,28.0,14.0,0.0
16,249073,1.0,0.0,0.0
19,996074,20.0,47.0,24.0
21,497959,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2490388,25.0,53.0,5.0
1,15,249070,28.0,14.0,0.0
2,16,249073,1.0,0.0,0.0
3,19,996074,20.0,47.0,24.0
4,21,497959,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,25.0,53.0,5.0
1,15,28.0,14.0,0.0
2,16,1.0,0.0,0.0
3,19,20.0,47.0,24.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,25.0
1,15,Released,28.0
2,16,Released,1.0
3,19,Released,20.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,5.0,0.0,0.0,24.0,0.0
Executing,53.0,14.0,0.0,47.0,1.0
Released,25.0,28.0,1.0,20.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,5.0,0.0,0.0,24.0,0.0
1,Executing,53.0,14.0,0.0,47.0,1.0
2,Released,25.0,28.0,1.0,20.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,5.0,0.0,0.0,24.0,0.0
1,Executing,53.0,14.0,0.0,47.0,1.0
2,Released,25.0,28.0,1.0,20.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()